# Lesson 6 - Schema Proposal for Structured Data

In this lesson, you will build an agent that can propose a knowledge graph schema.
The agent will analyze the approved import files and consider how to satisfy the approved user goal.

You'll learn:
- how to pair agents into a "critic pattern"
- how to use tools to focus an agent's thinking


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>To access the helper.py, neo4j_for_adk.py and tools.py files :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

</div>

## 6.1. Agent


<img src="images/entire_solution.png" width="500">

Multiple agents will collaborate to propose construction rules for a knowledge graph

- Input: `approved_user_goal`, `approved_files`
- Output: `approved_construction_plan`
- Tools: `get_approved_user_goal`, `get_approved_files`, `sample_file`, 
        `propose_node_construction`, `propose_relationship_construction`, 
        `get_proposed_construction_plan`,`approve_proposed_construction_plan`


**Workflow**


This workflow will use multiple agents to propose a construction plan for the knowledge graph.

A top-level 'schema_refinement_loop' will coordinate three agents operating in a loop:
1. A 'schema_proposal_agent' that proposes a schema and construction plan for the knowledge graph.
2. A 'schema_critic_agent' that critiques the proposed schema and construction plan for the knowledge graph.
3. A 'CheckStatusAndEscalate' agent that checks the feedback of the critic agent

## 6.2. Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

## 6.3. Agent Instructions for Schema Proposal

## 6.4. Tool Definitions for Schema Proposal

## 6.5. Define the Agent for Schema Proposal

### Try the Agent for Schema Proposal

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by LLMs can vary with each execution due to their stochastic nature. Your results might differ from those shown in the video.</p>

## 6.6. Agent Instructions for Schema Critic

## 6.7. Tool Definitions for Schema Critic

The schema critic has read-only tools so that it can not make any changes directly, only suggest changes.

## 6.8. Define the Agent for Schema Critic

The result of the schema critic is placed automatically in the `feedback` key of the context state.

## 6.9. Define the refinement loop

### CheckStatusAndEscalate agent

### Loop agent

### Try the refinement loop

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by LLMs can vary with each execution due to their stochastic nature. Your results might differ from those shown in the video.</p>

## 6.11. Optional - Sequence diagram illustrating the workflow of  "Refinement Loop Agent"  

<img src="images/schema_proposal_structured_sequence.png" width="600">

## 6.12. Extra - Create the top-level agent

The top-level agent will manage collaboration with the user and coordinates the work
of the sub-agents by calling them as a tool.

```python

from google.adk.tools import agent_tool
from google.adk.agents.callback_context import CallbackContext


schema_proposal_coordinator_instruction = """
    You are a coordinator for the schema proposal process. Use tools to propose a schema to the user.
    If the user disapproves, use the tools to refine the schema and ask the user to approve again.
    If the user approves, use the 'approve_proposed_schema' tool to record the approval.
    When the schema approval has been recorded, use the 'finished' tool.

    Guidance for tool use:
    - Use the 'schema_refinement_loop' tool to produce or update a proposed schema with construction rules. 
    - Use the 'get_proposed_schema' tool to get the proposed schema
    - Use the 'get_proposed_construction_plan' tool to get the construction rules for transforming approved files into the schema
    - Present the proposed schema and construction rules to the user for approval
    - If they disapprove, consider their feedback and go back to step 1
    - If the user approves, use the 'approve_proposed_schema' tool and the 'approve_proposed_construction_plan' tool to record the approval
"""

refinement_loop_as_tool = agent_tool.AgentTool(schema_refinement_loop)


# initialize context with blank feedback, which may get filled later by the schema_critic_agent
def initialize_feedback(callback_context: CallbackContext) -> None:
    callback_context.state["feedback"] = ""

schema_proposal_coordinator = LlmAgent(
    name="schema_proposal_coordinator",
    model=llm,
    instruction=schema_proposal_coordinator_instruction,
    tools=[
        refinement_loop_as_tool, 
        get_proposed_construction_plan, 
        approve_proposed_construction_plan
    ], 
    before_agent_callback=initialize_feedback
)

structured_schema_proposal_caller = await make_agent_caller(schema_proposal_coordinator, {
    "feedback": "",
    "approved_user_goal": {
        "kind_of_graph": "supply chain analysis",
        "description": "A multi-level bill of materials for manufactured products, useful for root cause analysis.."
    },
    "approved_files": [
        'assemblies.csv', 
        'parts.csv', 
        'part_supplier_mapping.csv', 
        'products.csv', 
        'suppliers.csv'
    ]
})

# Run the Initial Conversation
await structured_schema_proposal_caller.call("How can these files be imported?")

session_end = await structured_schema_proposal_caller.get_session()
print("Session state: ", session_end.state)

# Agree with the file suggestions
await structured_schema_proposal_caller.call("Yes, let's do it!", True)

session_end = await structured_schema_proposal_caller.get_session()

print("Approved construction plan: ", session_end.state['approved_user_goal'])

```